In [1]:
from bubls.utils.rag_design.rag_building_blocks import RAGBuildingBlocks
import logging
import sys
from metadata import lyft_10k, wiki_public_companies, papers_ridesharing

<jemalloc>: Unsupported system page size


In [2]:
logging.basicConfig(
    stream=sys.stdout,
    level=logging.ERROR,
    format="%(asctime)s - %(levelname)s - %(message)s",
)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
components_cfg = {
    "lyft_10k": lyft_10k,
    "wiki_public_companies": wiki_public_companies,
    "papers_ridesharing": papers_ridesharing,
}

In [4]:
rag = RAGBuildingBlocks(components_cfg)
rag.execute()

Loading data artifacts for lyft_10k
Loading LLM eval data for lyft_10k, train
Loading LLM eval data for lyft_10k, val
Loading LLM eval data for lyft_10k, test
Generating Nodes for lyft_10k, train
Generating Nodes for lyft_10k, val
Generating Nodes for lyft_10k, test
Loading QA pairs for lyft_10k, train
Loading QA pairs for lyft_10k, val
Loading QA pairs for lyft_10k, test
Generating engines for lyft_10k
Generating Index for lyft_10k
Generating Query Engine for lyft_10k
Generating Query Engine Tool for lyft_10k
Generating Retriever for lyft_10k
Loading data artifacts for wiki_public_companies
Loading LLM eval data for wiki_public_companies, train
Loading LLM eval data for wiki_public_companies, val
Loading LLM eval data for wiki_public_companies, test
Generating Nodes for wiki_public_companies, train
Generating Nodes for wiki_public_companies, val
Generating Nodes for wiki_public_companies, test
Loading QA pairs for wiki_public_companies, train
Loading QA pairs for wiki_public_companies

In [6]:
a=rag.query_engine["lyft_10k"]
a.

## Fine-tune embeddings

## Agent

In [5]:
from bubls.utils.agents.finance_tools import FinanceTools
from llama_index.core.agent import ReActAgent
from llama_index.core.agent import FunctionCallingAgentWorker, AgentRunner
from llama_index.packs.agents_coa import CoAAgentWorker
from llama_index.agent.openai import OpenAIAgent

finance_tool = FinanceTools()
finance_tools_list = finance_tool.to_tool_list()
agent = OpenAIAgent.from_tools(
    finance_tools_list + rag.query_engine_tools,
    # llm=OpenAI(model="gpt-4"),
    verbose=True
)
# agent = AgentRunner(agent)

# response = agent.chat("""
#     Without using previous knowledge and only the toold provided.
#     Tell me the nmext information about Nvidia:
#     - Who have been the CEOs of Nvidia.
#     - Plot the open and close prices of Nvidia for the last 3 months.
#     - Stock information of the last 2 days.             
# """)
response = agent.chat("""
    Can you give me the latest Nvidia news for the last month.             
""")
print(response)

## Evaluate

In [15]:
from bubls.utils.evaluation.evaluate_llm import evaluate_llm_performance
from bubls.utils.evaluation.evaluate_embeddings import evaluate_retriever
evaluate_llm_performance(rag.llm_eval_data["papers_ridesharing"]["val"], rag.query_engine["papers_ridesharing"])

  0%|          | 0/15 [00:00<?, ?it/s]

{'response_time': 2.9110182444254558,
 'faithfulness': 0.9333333333333333,
 'relevancy': 0.9333333333333333}

In [1]:
a=evaluate_retriever(rag.qa_pairs["papers_ridesharing"], rag.retriever["papers_ridesharing"])
print(a.is_hit.mean(), a.mrr.mean())

NameError: name 'evaluate_retriever' is not defined

In [29]:
a=rag.qa_pairs["lyft_10k"]
a.query_docid_pairs[0]

("How does the retention of stockholders' ownership proportions before and after a change in ownership impact the total voting power of the stock of the Company or its ultimate parent entity?",
 ['e6ef1460-0e63-46ef-8a74-f8b38cd865b9'])

In [34]:
a.corpus

{'e6ef1460-0e63-46ef-8a74-f8b38cd865b9': "Further,\xa0if\xa0the\xa0stockholders\xa0of\xa0the\xa0Company immediately\n\xa0before \xa0such \xa0change \xa0in \xa0ownership \xa0continue \xa0to \xa0retain \xa0immediately \xa0after \xa0the \xa0change \xa0in \xa0ownership, \xa0insubstantially\n\xa0the\xa0same\xa0proportions\xa0as\xa0their\xa0ownership\xa0of\xa0shares\xa0of\xa0the\xa0Company's\xa0voting\xa0stock\xa0immediately\xa0prior\xa0to\xa0thechange\n\xa0in\xa0ownership,\xa0direct\xa0or\xa0indirect\xa0beneficial\xa0ownership\xa0of\xa0fifty\xa0percent\xa0(50%)\xa0or\xa0more\xa0of\xa0the\xa0total\xa0voting\xa0power\xa0ofthe\xa0stock\xa0of\xa0the\xa0Company\xa0or\xa0of\xa0the\xa0ultimate\xa0parent\xa0entity\xa0of\xa0the\xa0Company,\xa0such\xa0event",
 '9b373f4f-2cb2-470d-bb6f-3c4c661838d8': 'Any of the foregoing risks could also result in decreased usage of our network of Light Vehicles and adversely affect our business, brand,financial conditions and \nresults of operations.If we fail to ef